In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
#from pytorchtools import EarlyStopping
from tqdm.notebook import tqdm
import time
import datetime
import io
import random
import statistics
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from statistics import mean
from collections import Counter

def seed_everything(seed=0):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(230)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/Data

In [ ]:
df1 = pd.read_csv('sample_2_1_F.csv')
df2 = pd.read_csv('sample_2_2_F.csv')
df3 = pd.read_csv('sample_2_3_F.csv')
df4 = pd.read_csv('sample_2_4_F.csv')

In [ ]:
# Define the custom dataset
class CustomDataset(Dataset):
    def __init__(self, X, Y, sequence_length, forward_step):
        self.X = X
        self.Y = Y
        self.sequence_length = sequence_length
        self.forward_step = forward_step

    def __len__(self):
        return len(self.X)

    def pad_sequence(self, x, seq_length):
        diff = seq_length - x.shape[0]
        x = F.pad(x, (0, 0, 0, diff))
        return x

    def __getitem__(self, idx):
        idx = min(idx*self.forward_step, len(self.X)-1 - self.sequence_length)
        x = torch.from_numpy(self.X[idx : idx + self.sequence_length]).type(torch.float32)
        x = self.pad_sequence(x, self.sequence_length)

        y = torch.from_numpy(self.Y[idx : idx + self.sequence_length]).type(torch.long)

        if (y == 1).sum().item() >= 2:
            y = y.any().type(torch.long)
        else:
            y = torch.tensor(0, dtype=torch.long)

        return x, y, torch.from_numpy(self.Y[idx : idx + self.sequence_length]).type(torch.long), self.X[idx : idx + self.sequence_length]

# Define the RNN model
class DeepRNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout_prob):
        super(DeepRNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout_prob = dropout_prob
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        hidden = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, hidden = self.rnn(x.to(device), hidden.to(device))
        out = self.dropout(out)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

# Define the LSTM model
class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout_prob):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout_prob = dropout_prob
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)  # Initial hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) # Initial cell state
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out)
        out = out[:, -1, :]
        out = self.fc(out)  # Use only the last time step's output for classification
        return out

# Define the GRU model
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout_prob):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout_prob = dropout_prob
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)  # Initial hidden state
        # Forward pass through GRU layer
        out, h = self.gru(x, h)
        out = self.dropout(out)
        out = out[:, -1, :]
        # Forward pass through the linear layer
        out = self.fc(out)
        return out

In [ ]:
df = pd.concat([df1, df3, df2, df4], ignore_index=True)

In [ ]:
# Encode data
le_y = LabelEncoder()

X = df.loc[:, ['Temps','Source', 'From', 'Call-ID', 'Contact']].apply(le_y.fit_transform)

def label_encoder(x):
    return x == 'Attack'

Y = df.loc[:, ['Label']].apply(label_encoder).astype('int')

# Split data
X_train  = X.loc[len(df1)+1+len(df3) : ,:].values
y_train  = Y.loc[len(df1)+1+len(df3) : ,:].values

X_val  = X.loc[:len(df1) ,:].values
y_val  = Y.loc[:len(df1) ,:].values

X_test  = X.loc[ : len(df1)+len(df3) ,:].values
y_test  = Y.loc[ : len(df1)+len(df3),:].values

df11 = pd.concat([df1, df3], ignore_index=True)

df11['Source'] = df11['Source'].str.replace('192.168.56', '199.162.59')
df11['From'] = df11['From'].str.replace('192.168.56', '199.162.59')
df11['Call-ID'] = df11['Call-ID'].str.replace('192.168.56', '199.162.59')
df11['Contact'] = df11['Contact'].str.replace('192.168.56', '199.162.59')

# Encode data
le_y = LabelEncoder()

X1 = df11.loc[:, ['Temps','Source', 'From', 'Call-ID', 'Contact']].apply(le_y.fit_transform)

Y1 = df11.loc[:, ['Label']].apply(label_encoder).astype('int')

X1_test  = X1.values
y1_test  = Y1.values

In [ ]:
# Window size and forword step
sequence_length = 10
forward_step = 2
forward_step_inf = 2

# Create the training-validation dataset and dataloader
batch_size = 240
training_dataset = CustomDataset(X_train, y_train, sequence_length, forward_step)
train_dataloader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True)

validation_dataset = CustomDataset(X_val, y_val, sequence_length, forward_step)
val_dataloader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

test_dataset = CustomDataset(X_test, y_test, sequence_length, forward_step_inf)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

test_dataset_Bis = CustomDataset(X1_test, y1_test, sequence_length, forward_step_inf)
test_dataloader_Bis = DataLoader(test_dataset_Bis, batch_size=batch_size, shuffle=False)

In [ ]:
labels = []
for x, y, Y, X in train_dataloader:
    labels.extend(list(y.detach().cpu().numpy()))
    if np.unique(labels, return_counts=True)[1][1]==92315:
      break

values, counts = np.unique(labels, return_counts=True)
print(values, counts, counts[1]+counts[0])
print(counts[0] / np.sum(counts), counts[1] / np.sum(counts))

labels = []
for x, y, Y, X in val_dataloader:
    labels.extend(list(y.detach().cpu().numpy()))

values, counts = np.unique(labels, return_counts=True)
print(values, counts, counts[1]+counts[0])
print(counts[0] / np.sum(counts), counts[1] / np.sum(counts))

labels = []
for x, y, Y, X in test_dataloader:
    labels.extend(list(y.detach().cpu().numpy()))

values, counts = np.unique(labels, return_counts=True)
print(values, counts, counts[1]+counts[0])
print(counts[0] / np.sum(counts), counts[1] / np.sum(counts))

In [ ]:
print ("Train")
for x, y, Y, X in train_dataloader:
    print(x.shape)
    break

print ("Validation")
for x, y, Y, X in val_dataloader:
    print(x.shape)
    break

print ("Test")
for x, y, Y, X in test_dataloader:
    print(x.shape)
    break

In [ ]:
#df11

In [ ]:
def dessiner_graphe(tab1, tab2):
    indices1 = range(len(tab1))
    indices2 = range(len(tab2))

    valeurs1 = tab1
    valeurs2 = tab2

    plt.figure(figsize=(25, 6))
    plt.plot(indices1, valeurs1, 'bo-', label='True', color='blue')  # 'bo-' représente des points bleus reliés par des lignes
    plt.plot(indices2, valeurs2, 'bo-', label='Predicted', color='red')
    plt.xlabel('Indices')
    plt.ylabel('Valeurs')
    plt.title('Graphe des valeurs du tableau')
    plt.grid(True)
    plt.show()

In [ ]:
def compare_elements(table1, table2):
    different_indices = []

    for i in range(len(table1)):
        if table1[i] != table2[i]:
            different_indices.append(i)

    return different_indices

LSTM

In [ ]:
# Create empty lists to store accuracy and loss
train_acc = []
train_loss = []
val_acc_list = []
val_loss_list = []

# Define the hyperparameters
Nbr_row, Nbr_col = X_train.shape

input_size = Nbr_col
hidden_size = pow(2, 4) # Number of hidden units
output_size = 2 # Number of output classes
num_layers = sequence_length # Number of hidden layers
learning_rate = 0.0001
num_epochs = 10
drop_out = 0.10

# Define early stopping parameters
patience = 3 #3
best_loss = float('inf')
early_stop = False
epochs_no_improve = 0

# Move the model to the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(input_size, hidden_size, output_size, num_layers, drop_out).to(device)

model = nn.DataParallel(model).to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

since = time.time()

m = 'LSTM'

# Train the model
for epoch in range(num_epochs):
    total_loss = 0.0
    total_correct = 0
    total = 0

    total_val_loss = 0.0
    total_val_correct = 0
    total_val = 0

    if early_stop:
        # torch.save(model.state_dict(), '/home/ubuntu/Bureau/collab/model_Seq/'+ dt_string +'/'+ m +'_Epoch-'+ str(epoch) +'_best_model.pt')
        break

    pbar = tqdm(train_dataloader)

    # Training loop
    model.train()
    for x, y, Y, X in pbar:
        # Forward pass
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        outputs = model(x)
        # loss = criterion(outputs, y[:, 0].to(device))
        loss = criterion(outputs, y)
        total_loss += loss / batch_size

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Forward pass
        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == y).sum().item()
        total += y.size(0)

        # Calculate accuracy and loss for the epoch
        acc = total_correct / total
        loss = total_loss / len(train_dataloader)

        pbar.set_description('Epoch [{}/{}], Loss: {:.5f}, Acc: {:.5f}'.format(epoch+1, num_epochs, total_loss.item(), acc))

    # Append accuracy and loss to the lists
    train_acc.append(acc)
    train_loss.append(loss.item())

    # Validation loop
    model.eval()
    with torch.no_grad():
        for x, y, Y, X in val_dataloader:
            x = x.to(device)
            y = y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_val_loss += loss / batch_size

            _, predicted = torch.max(outputs.data, 1)
            total_val_correct += (predicted == y).sum().item()
            total_val += y.size(0)

        val_acc = total_val_correct / total_val
        val_loss = total_val_loss / len(val_dataloader)

        print('Epoch [{}/{}], Validation Loss: {:.5f}, Validation Acc: {:.5f}'.format(epoch+1, num_epochs, val_loss.item(), val_acc ))

    val_acc_list.append(val_acc)
    val_loss_list.append(val_loss.item())

    # Early stopping check
    if val_loss < best_loss:
        best_loss = val_loss
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve == patience:
            early_stop = True

time_elapsed = time.time() - since

print('============== FIN ==============')
print('\n\nTraining complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('=== Train Loss: {:.5f}, Train Acc: {:.5f}'.format( loss, acc ))
print('=== Validation Loss: {:.5f}, Validation Acc: {:.5f}'.format( val_loss, val_acc ))

In [ ]:
total_correct = 0
total_loss = 0
total_samples = 0

true_labels = []
predicted_labels = []

le_y_1 = LabelEncoder()
X11 = df.loc[:, ['Source']].apply(le_y_1.fit_transform)

i = 0

L = []
# Evaluate the model on the test dataset
model.eval()
j=0
with torch.no_grad():
    for x, y, Y, X in test_dataloader:

        x = x.to(device)
        y = y.to(device)
        outputs = model(x)

        loss = criterion(outputs, y)
        total_loss += loss / batch_size

        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == y).sum().item()
        total_samples += y.size(0)

        # Convert tensors to numpy arrays
        true_labels.extend(y.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

        comparison_result = y.cpu().numpy() == predicted.cpu().numpy()
        if False in comparison_result:
            i = 0
            print(compare_elements(y.cpu().numpy(), predicted.cpu().numpy()))
            for i in compare_elements(y.cpu().numpy(), predicted.cpu().numpy()):
                # Convert tensor to matrix
                matrix = np.matrix(X[i])

                # Extract the second column
                second_column = matrix[:, 1]

                # Convert the second column to a NumPy array
                second_column_array = np.array(second_column)
                #print(le_y_1.inverse_transform(second_column_array))
                print(Y[i])
                L.append(np.array(Y[i].T).flatten())

            j += len(compare_elements(y.cpu().numpy(), predicted.cpu().numpy()))

    print(j)


In [ ]:
array_counts = Counter(map(tuple, L))
sorted_results = dict(sorted(array_counts.items(), key=lambda item: item[1], reverse=True))
len(array_counts), array_counts, sorted_results

In [ ]:
total_correct = 0
total_loss = 0
total_samples = 0

true_labels = []
predicted_labels = []

le_y_1 = LabelEncoder()
X11 = df11.loc[:, ['Source']].apply(le_y_1.fit_transform)

i = 0

L = []
# Evaluate the model on the test dataset
model.eval()
j=0
with torch.no_grad():
    for x, y, Y, X in test_dataloader_Bis:

        x = x.to(device)
        y = y.to(device)
        outputs = model(x)

        loss = criterion(outputs, y)
        total_loss += loss / batch_size

        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == y).sum().item()
        total_samples += y.size(0)

        # Convert tensors to numpy arrays
        true_labels.extend(y.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

        comparison_result = y.cpu().numpy() == predicted.cpu().numpy()
        if False in comparison_result:
            i = 0
            print(compare_elements(y.cpu().numpy(), predicted.cpu().numpy()))
            for i in compare_elements(y.cpu().numpy(), predicted.cpu().numpy()):
                # Convert tensor to matrix
                matrix = np.matrix(X[i])

                # Extract the second column
                second_column = matrix[:, 1]

                # Convert the second column to a NumPy array
                second_column_array = np.array(second_column)
                #print(le_y_1.inverse_transform(second_column_array))
                print(Y[i])
                L.append(np.array(Y[i].T).flatten())

            j += len(compare_elements(y.cpu().numpy(), predicted.cpu().numpy()))

    print(j)


In [ ]:
array_counts = Counter(map(tuple, L))
sorted_results = dict(sorted(array_counts.items(), key=lambda item: item[1], reverse=True))
len(array_counts), array_counts, sorted_results

In [ ]:
total_correct = 0
total_loss = 0
total_samples = 0

true_labels = []
predicted_labels = []

le_y_1 = LabelEncoder()
X11 = df.loc[:, ['Source']].apply(le_y_1.fit_transform)

i = 0

since = time.time()
Seq_Time = []

# Evaluate the model on the test dataset
model.eval()
j=0
with torch.no_grad():
    for x, y, Y, X in test_dataloader:
        x = x.to(device)
        y = y.to(device)
        outputs = model(x)

        loss = criterion(outputs, y)
        total_loss += loss / batch_size

        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == y).sum().item()
        total_samples += y.size(0)

        # Convert tensors to numpy arrays
        true_labels.extend(y.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

        comparison_result = y.cpu().numpy() == predicted.cpu().numpy()
        if False in comparison_result:
            i = 0
            print(compare_elements(y.cpu().numpy(), predicted.cpu().numpy()))
            for i in compare_elements(y.cpu().numpy(), predicted.cpu().numpy()):
                # Convert tensor to matrix
                matrix = np.matrix(X[i])

                # Extract the second column
                second_column = matrix[:, 1]

                # Convert the second column to a NumPy array
                second_column_array = np.array(second_column)
                print(le_y_1.inverse_transform(second_column_array))
                print(Y[i])

            dessiner_graphe(y.cpu().numpy(), predicted.cpu().numpy())
            j += len(compare_elements(y.cpu().numpy(), predicted.cpu().numpy()))

    print(j)

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate the test accuracy and loss
test_accuracy = total_correct / total_samples
test_loss = total_loss / total_samples

print('=== Test Loss: {:.5f}, Test Acc: {:.5f}'.format(test_loss, test_accuracy ))

# Decode the predicted and actual labels
label_decoder = {0: 'Good',  1: 'Attack'}

y_pred = [label_decoder[i] for i in predicted_labels]
y_true = [label_decoder[i] for i in true_labels]

# Generate the IoU
cm = metrics.confusion_matrix(y_true, y_pred, normalize='true')
print('=== IoU: {:.5f}'.format(cm.ravel()[3]/(cm.ravel()[3]+cm.ravel()[2]+cm.ravel()[1])))

# Generate the classification report
report = classification_report(y_true, y_pred, digits=5)
print("Classification Report:")
print(report)

# Generate the confusion matrix
print("Confusion Matrix:")
print(cm)

sns.heatmap(cm, annot=True, xticklabels=['Good', 'Attack'], yticklabels=['Good', 'Attack'], fmt='.5f')
plt.title('LSTM')

In [ ]:
total_correct = 0
total_loss = 0
total_samples = 0

true_labels = []
predicted_labels = []

le_y_1 = LabelEncoder()
X11 = df11.loc[:, ['Source']].apply(le_y_1.fit_transform)

i = 0

# Evaluate the model on the test dataset
model.eval()
j=0
with torch.no_grad():
    for x, y, Y, X in test_dataloader_Bis:

        x = x.to(device)
        y = y.to(device)
        outputs = model(x)

        loss = criterion(outputs, y)
        total_loss += loss / batch_size

        _, predicted = torch.max(outputs.data, 1)
        total_correct += (predicted == y).sum().item()
        total_samples += y.size(0)

        # Convert tensors to numpy arrays
        true_labels.extend(y.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

        comparison_result = y.cpu().numpy() == predicted.cpu().numpy()
        if False in comparison_result:
            i = 0
            print(compare_elements(y.cpu().numpy(), predicted.cpu().numpy()))
            for i in compare_elements(y.cpu().numpy(), predicted.cpu().numpy()):
                # Convert tensor to matrix
                matrix = np.matrix(X[i])

                # Extract the second column
                second_column = matrix[:, 1]

                # Convert the second column to a NumPy array
                second_column_array = np.array(second_column)
                print(le_y_1.inverse_transform(second_column_array))
                print(Y[i])

            dessiner_graphe(y.cpu().numpy(), predicted.cpu().numpy())
            j += len(compare_elements(y.cpu().numpy(), predicted.cpu().numpy()))

    print(j)

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predicted_labels = np.array(predicted_labels)

# Calculate the test accuracy and loss
test_accuracy = total_correct / total_samples
test_loss = total_loss / total_samples

print('=== Test Loss: {:.5f}, Test Acc: {:.5f}'.format(test_loss, test_accuracy ))

# Decode the predicted and actual labels
label_decoder = {0: 'Good',  1: 'Attack'}

y_pred = [label_decoder[i] for i in predicted_labels]
y_true = [label_decoder[i] for i in true_labels]

# Generate the IoU
cm = metrics.confusion_matrix(y_true, y_pred, normalize='true')
print('=== IoU: {:.5f}'.format(cm.ravel()[3]/(cm.ravel()[3]+cm.ravel()[2]+cm.ravel()[1])))

# Generate the classification report
report = classification_report(y_true, y_pred, digits=5)
print("Classification Report:")
print(report)

# Generate the confusion matrix
print("Confusion Matrix:")
print(cm)

sns.heatmap(cm, annot=True, xticklabels=['Good', 'Attack'], yticklabels=['Good', 'Attack'], fmt='.5f')
plt.title('LSTM')

In [ ]:
test_dataset_Inf = CustomDataset(X_test, y_test, sequence_length, forward_step_inf)
test_dataloader_Inf = DataLoader(test_dataset_Inf, batch_size=1, shuffle=False)

Seq_Time = []

# Evaluate the model on the test dataset
model.eval()
j=0
with torch.no_grad():
    for x, y, Y, X in test_dataloader_Inf:
        x = x.to(device)
        y = y.to(device)

        since_Test_Seq = time.time()
        outputs = model(x)
        Seq_Time.append(time.time() - since_Test_Seq)

execution_time_ms = mean(Seq_Time) * 1000
execution_time = datetime.timedelta(milliseconds=execution_time_ms)
minutes, seconds = divmod(execution_time.seconds, 60)
milliseconds = execution_time.microseconds // 1000
microseconds = execution_time.microseconds % 1000
print('mean Time Seq_Test (Seq = {:1.0f} msg) in {:.2f}m {:.2f}s {:.4f}ms {:.4f}µs'.format(sequence_length, minutes, seconds, milliseconds, microseconds))

In [ ]:
test_dataset_Inf = CustomDataset(X_test, y_test, sequence_length, forward_step_inf)
test_dataloader_Inf = DataLoader(test_dataset_Inf, batch_size=batch_size, shuffle=False)

Seq_Time = []

# Evaluate the model on the test dataset
model.eval()
j=0
with torch.no_grad():
    for x, y, Y, X in test_dataloader_Inf:
        x = x.to(device)
        y = y.to(device)

        since_Test_Seq = time.time()
        outputs = model(x)
        Seq_Time.append(time.time() - since_Test_Seq)

execution_time_ms = mean(Seq_Time) * 1000
execution_time = datetime.timedelta(milliseconds=execution_time_ms)
minutes, seconds = divmod(execution_time.seconds, 60)
milliseconds = execution_time.microseconds // 1000
microseconds = execution_time.microseconds % 1000
print('mean Time Seq_Test (Seq = {:1.0f} msg) in {:.2f}m {:.2f}s {:.4f}ms {:.4f}µs'.format(sequence_length, minutes, seconds, milliseconds, microseconds))

In [ ]:
test_dataset_Inf = CustomDataset(X1_test, y1_test, sequence_length, forward_step_inf)
test_dataloader_Inf = DataLoader(test_dataset_Inf, batch_size=1, shuffle=False)

Seq_Time = []

# Evaluate the model on the test dataset
model.eval()
j=0
with torch.no_grad():
    for x, y, Y, X in test_dataloader_Inf:
        x = x.to(device)
        y = y.to(device)

        since_Test_Seq = time.time()
        outputs = model(x)
        Seq_Time.append(time.time() - since_Test_Seq)

execution_time_ms = mean(Seq_Time) * 1000
execution_time = datetime.timedelta(milliseconds=execution_time_ms)
minutes, seconds = divmod(execution_time.seconds, 60)
milliseconds = execution_time.microseconds // 1000
microseconds = execution_time.microseconds % 1000
print('mean Time Seq_Test (Seq = {:1.0f} msg) in {:.2f}m {:.2f}s {:.4f}ms {:.4f}µs'.format(sequence_length, minutes, seconds, milliseconds, microseconds))

In [ ]:
test_dataset_Inf = CustomDataset(X1_test, y1_test, sequence_length, forward_step_inf)
test_dataloader_Inf = DataLoader(test_dataset_Inf, batch_size=batch_size, shuffle=False)

Seq_Time = []

# Evaluate the model on the test dataset
model.eval()
j=0
with torch.no_grad():
    for x, y, Y, X in test_dataloader_Inf:
        x = x.to(device)
        y = y.to(device)

        since_Test_Seq = time.time()
        outputs = model(x)
        Seq_Time.append(time.time() - since_Test_Seq)

execution_time_ms = mean(Seq_Time) * 1000
execution_time = datetime.timedelta(milliseconds=execution_time_ms)
minutes, seconds = divmod(execution_time.seconds, 60)
milliseconds = execution_time.microseconds // 1000
microseconds = execution_time.microseconds % 1000
print('mean Time Seq_Test (Seq = {:1.0f} msg) in {:.2f}m {:.2f}s {:.4f}ms {:.4f}µs'.format(sequence_length, minutes, seconds, milliseconds, microseconds))

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/step 2 Bis
PATH = m + "_Bis_W"+str(sequence_length) +"-step"+ str(forward_step)+'.pt'
torch.save(model.state_dict(), PATH)